In [2]:
# mieyhgnaj set 🤪

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style("white")

import missingno as msno
import plotly.express as px
from matplotlib import cm
from matplotlib import style
from matplotlib import font_manager
from matplotlib import rc

rc('font', family='Arial Unicode MS')

import warnings
warnings.filterwarnings("ignore")

## 🚙 `타깃 컬럼을 제외한 컬럼은 SOCAR 측의 요청대로 비공개 처리!`

In [5]:
train = pd.read_csv('datas/train_encode_ver2.csv', index_col=0)
test = pd.read_csv('datas/test_encode_ver2.csv', index_col=0)

# Outlier Removal

In [7]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [8]:
rm_out = train[['x14', 'x15']]
rm_out

,x14,x15
0,130254.67800,0
1,1341009.16300,0
2,524030.56960,1209410
3,0.00000,1217280
4,0.00000,0
...,...,...
12874,667293.08700,0
12875,0.00000,0
12876,618439.44180,0
12877,0.00000,0


In [13]:
data = rm_out['x15']
data

0              0
1              0
2        1209410
3        1217280
4              0
          ...   
12874          0
12875          0
12876          0
12877          0
12878    1188750
Name: x15, Length: 12879, dtype: int64

### 1. Standard Deviation Method

In [14]:
# identify outliers with standard deviation
from numpy import mean
from numpy import std
# calculate summary statistics
data_mean, data_std = mean(data), std(data)
# identify outliers
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
# identify outliers
outliers = [x for x in data if x < lower or x > upper]
print('Identified outliers: %d' % len(outliers))
# remove outliers
outliers_removed = [x for x in data if x >= lower and x <= upper]
print('Non-outlier observations: %d' % len(outliers_removed))

Identified outliers: 180
Non-outlier observations: 12699


### 2. Interquartile Range Method

In [15]:
# identify outliers with interquartile range
from numpy import percentile
# calculate interquartile range
q25, q75 = percentile(data, 25), percentile(data, 75)
iqr = q75 - q25
print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))
# calculate the outlier cutoff
cut_off = iqr * 1.5
lower, upper = q25 - cut_off, q75 + cut_off
# identify outliers
outliers = [x for x in data if x < lower or x > upper]
print('Identified outliers: %d' % len(outliers))
# remove outliers
outliers_removed = [x for x in data if x >= lower and x <= upper]
print('Non-outlier observations: %d' % len(outliers_removed))

Percentiles: 25th=0.000, 75th=554645.000, IQR=554645.000
Identified outliers: 1800
Non-outlier observations: 11079


In [16]:
data.describe()

count      12879.00000
mean      837754.65184
std      3063805.65025
min            0.00000
25%            0.00000
50%            0.00000
75%       554645.00000
max     94714350.00000
Name: x15, dtype: float64

In [17]:
# Standard Deviation Method로 해봐야겠다

In [276]:
mean(outliers)

5029615.182222222

In [278]:
mean(outliers_removed)

156705.01245599784

In [18]:
# identify outliers with standard deviation
from numpy import mean
from numpy import std

def rm_outlier(data):
    # calculate summary statistics
    data_mean, data_std = mean(data), std(data)
    # identify outliers
    cut_off = data_std * 3
    lower, upper = data_mean - cut_off, data_mean + cut_off

    rm_out_data = pd.DataFrame(data)
    for idx, x in enumerate(data):
        if x < lower or x > upper:
            rm_out_data.loc[idx] = 0
#             rm_out_data.drop(idx, inplace=True)

    # identify outliers
    outliers = [x for x in data if x < lower or x > upper]
    print('Identified outliers: %d' % len(outliers))
    # remove outliers
    outliers_removed = [x for x in data if x >= lower and x <= upper]
    print('Non-outlier observations: %d' % len(outliers_removed))
    
    return rm_out_data

In [ ]:
cost_mean['x14'].replace(0, statistics.mean(cost_mean['x14']), inplace=True)

In [20]:
mean(train['x15'])

837754.6518363227

In [22]:
rm_x15 = rm_outlier(rm_out['x15'])
rm_x15

Identified outliers: 180
Non-outlier observations: 12699


,x15
0,0
1,0
2,1209410
3,1217280
4,0
...,...
12874,0
12875,0
12876,0
12877,0


In [23]:
rm_x14 = rm_outlier(rm_out['x14'])
rm_x14

Identified outliers: 173
Non-outlier observations: 12706


,x14
0,130254.67800
1,1341009.16300
2,524030.56960
3,0.00000
4,0.00000
...,...
12874,667293.08700
12875,0.00000
12876,618439.44180
12877,0.00000


### outlier 제거한 버전으로 교체

In [24]:
train_out = train.copy()

In [25]:
train_out['x15'] = rm_x15
train_out['x14'] = rm_x14
train_out

,fraud_YN,x2,x4,x5,x7,x8,x9,x10,x14,x15,...,x16_0,x16_1,x16_2,x16_3,x16_4,x16_5,x20_0,x20_1,x20_2,x20_3
0,0,1,0,2,100,2,0,0,130254.67800,0,...,0,1,0,0,0,0,1,0,0,0
1,0,1,0,4,0,2,0,0,1341009.16300,0,...,0,0,0,1,0,0,1,0,0,0
2,0,0,0,1,100,1,0,0,524030.56960,1209410,...,0,1,0,0,0,0,1,0,0,0
3,1,1,0,4,100,3,0,0,0.00000,1217280,...,1,0,0,0,0,0,0,1,0,0
4,0,0,0,4,0,1,0,0,0.00000,0,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12874,0,0,1,4,100,1,0,0,667293.08700,0,...,1,0,0,0,0,0,1,0,0,0
12875,0,0,0,2,100,2,0,0,0.00000,0,...,0,1,0,0,0,0,1,0,0,0
12876,0,0,1,4,100,1,1,0,618439.44180,0,...,0,0,1,0,0,0,1,0,0,0
12877,0,0,0,2,100,2,0,0,0.00000,0,...,0,0,0,0,1,0,1,0,0,0


In [26]:
train_out.to_csv('datas/train_out.csv')